# Decorators in Python (OOP Context)

---

## Table of Contents
1. Decorator Basics Recap
2. Class Method Decorators
3. @staticmethod Decorator
4. @classmethod Decorator
5. @property Decorator (Deep Dive)
6. Class Decorators
7. Decorating Methods with Arguments
8. Method Decorator Use Cases
9. Combining Multiple Decorators
10. Practical Examples
11. Key Points
12. Practice Exercises

---

## 1. Decorator Basics Recap

A **decorator** is a function that takes another function and extends its behavior without modifying it.

In [ ]:
# Basic decorator pattern
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Before calling {func.__name__}")
        result = func(*args, **kwargs)
        print(f"After calling {func.__name__}")
        return result
    return wrapper

@my_decorator
def greet(name):
    return f"Hello, {name}!"

# This is equivalent to: greet = my_decorator(greet)
print(greet("Alice"))

In [ ]:
# Preserving function metadata with functools.wraps
from functools import wraps

def better_decorator(func):
    @wraps(func)  # Preserves __name__, __doc__, etc.
    def wrapper(*args, **kwargs):
        """Wrapper function"""
        return func(*args, **kwargs)
    return wrapper

@better_decorator
def example():
    """This is the example docstring."""
    pass

print(f"Function name: {example.__name__}")
print(f"Function doc: {example.__doc__}")

---

## 2. Class Method Decorators

Decorators can be applied to class methods to add functionality.

In [ ]:
from functools import wraps

def log_method(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        print(f"Calling {func.__name__} on {self.__class__.__name__}")
        result = func(self, *args, **kwargs)
        print(f"Result: {result}")
        return result
    return wrapper

class Calculator:
    @log_method
    def add(self, a, b):
        return a + b
    
    @log_method
    def multiply(self, a, b):
        return a * b

calc = Calculator()
calc.add(3, 5)
print()
calc.multiply(4, 7)

In [ ]:
# Timing decorator for methods
import time
from functools import wraps

def timed_method(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        start = time.time()
        result = func(self, *args, **kwargs)
        elapsed = time.time() - start
        print(f"{self.__class__.__name__}.{func.__name__}: {elapsed:.4f}s")
        return result
    return wrapper

class DataProcessor:
    @timed_method
    def process(self, data):
        # Simulate processing
        time.sleep(0.1)
        return [x * 2 for x in data]

processor = DataProcessor()
result = processor.process([1, 2, 3, 4, 5])

---

## 3. @staticmethod Decorator

**Static methods** don't receive implicit first argument (no `self` or `cls`).

**Use when:**
- Method doesn't need access to instance or class
- Utility function that belongs to the class namespace
- Could be standalone function but logically belongs to class

In [ ]:
class MathUtils:
    @staticmethod
    def add(a, b):
        """Add two numbers."""
        return a + b
    
    @staticmethod
    def is_even(n):
        """Check if number is even."""
        return n % 2 == 0
    
    @staticmethod
    def factorial(n):
        """Calculate factorial."""
        if n <= 1:
            return 1
        return n * MathUtils.factorial(n - 1)

# Can call on class directly
print(f"MathUtils.add(3, 5) = {MathUtils.add(3, 5)}")
print(f"MathUtils.is_even(4) = {MathUtils.is_even(4)}")
print(f"MathUtils.factorial(5) = {MathUtils.factorial(5)}")

# Can also call on instance
utils = MathUtils()
print(f"utils.add(1, 2) = {utils.add(1, 2)}")

In [ ]:
# Practical example: Validator class
class Validator:
    @staticmethod
    def is_valid_email(email):
        import re
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(pattern, email))
    
    @staticmethod
    def is_valid_phone(phone):
        import re
        digits = re.sub(r'\D', '', phone)
        return len(digits) == 10
    
    @staticmethod
    def is_valid_password(password, min_length=8):
        if len(password) < min_length:
            return False
        has_upper = any(c.isupper() for c in password)
        has_lower = any(c.islower() for c in password)
        has_digit = any(c.isdigit() for c in password)
        return has_upper and has_lower and has_digit

print(f"Valid email: {Validator.is_valid_email('test@example.com')}")
print(f"Invalid email: {Validator.is_valid_email('invalid')}")
print(f"Valid phone: {Validator.is_valid_phone('123-456-7890')}")
print(f"Valid password: {Validator.is_valid_password('Pass123!')}")
print(f"Weak password: {Validator.is_valid_password('weak')}")

---

## 4. @classmethod Decorator

**Class methods** receive the class as first argument (`cls` instead of `self`).

**Use when:**
- Alternative constructors (factory methods)
- Need to access or modify class state
- Method should work correctly with inheritance

In [ ]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    def __repr__(self):
        return f"Date({self.year}, {self.month}, {self.day})"
    
    # Alternative constructor from string
    @classmethod
    def from_string(cls, date_string):
        """Create Date from 'YYYY-MM-DD' string."""
        year, month, day = map(int, date_string.split('-'))
        return cls(year, month, day)
    
    # Alternative constructor for today
    @classmethod
    def today(cls):
        """Create Date for today."""
        from datetime import date
        d = date.today()
        return cls(d.year, d.month, d.day)

# Standard constructor
d1 = Date(2024, 1, 15)
print(f"Standard: {d1}")

# Factory method from string
d2 = Date.from_string("2024-06-20")
print(f"From string: {d2}")

# Factory method for today
d3 = Date.today()
print(f"Today: {d3}")

In [ ]:
# Classmethod works correctly with inheritance
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}', {self.age})"
    
    @classmethod
    def from_dict(cls, data):
        return cls(data['name'], data['age'])

class Employee(Person):
    def __init__(self, name, age, employee_id=None):
        super().__init__(name, age)
        self.employee_id = employee_id

# cls is Person
person = Person.from_dict({'name': 'Alice', 'age': 30})
print(f"Person: {person}, type: {type(person).__name__}")

# cls is Employee - polymorphic!
employee = Employee.from_dict({'name': 'Bob', 'age': 25})
print(f"Employee: {employee}, type: {type(employee).__name__}")

In [ ]:
# Classmethod for tracking class-level state
class User:
    _user_count = 0
    _users = []
    
    def __init__(self, username):
        self.username = username
        User._user_count += 1
        User._users.append(self)
    
    @classmethod
    def get_user_count(cls):
        return cls._user_count
    
    @classmethod
    def get_all_users(cls):
        return cls._users.copy()
    
    @classmethod
    def reset(cls):
        cls._user_count = 0
        cls._users.clear()

u1 = User("alice")
u2 = User("bob")
u3 = User("charlie")

print(f"User count: {User.get_user_count()}")
print(f"All users: {[u.username for u in User.get_all_users()]}")

User.reset()
print(f"After reset: {User.get_user_count()}")

In [ ]:
# Comparison: staticmethod vs classmethod
class Demo:
    class_var = "class variable"
    
    def instance_method(self):
        """Has access to self (instance) and self.__class__ (class)"""
        return f"Instance: {self}, Class var: {self.class_var}"
    
    @classmethod
    def class_method(cls):
        """Has access to cls (class) but not instance"""
        return f"Class: {cls}, Class var: {cls.class_var}"
    
    @staticmethod
    def static_method():
        """Has no access to instance or class automatically"""
        return "Static: no automatic access"

obj = Demo()
print(obj.instance_method())
print(Demo.class_method())
print(Demo.static_method())

---

## 5. @property Decorator (Deep Dive)

**Property** creates managed attributes with getters, setters, and deleters.

In [ ]:
# Full property syntax
class Circle:
    def __init__(self, radius):
        self._radius = radius  # Private storage
    
    @property
    def radius(self):
        """Get the radius."""
        print("Getting radius")
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """Set the radius with validation."""
        print(f"Setting radius to {value}")
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value
    
    @radius.deleter
    def radius(self):
        """Delete the radius."""
        print("Deleting radius")
        del self._radius

c = Circle(5)
print(f"Radius: {c.radius}")

c.radius = 10
print(f"New radius: {c.radius}")

try:
    c.radius = -1
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
# Computed properties (read-only)
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def diameter(self):
        """Computed: diameter = 2 * radius"""
        return self.radius * 2
    
    @property
    def area(self):
        """Computed: area = pi * r^2"""
        return math.pi * self.radius ** 2
    
    @property
    def circumference(self):
        """Computed: circumference = 2 * pi * r"""
        return 2 * math.pi * self.radius

c = Circle(5)
print(f"Radius: {c.radius}")
print(f"Diameter: {c.diameter}")
print(f"Area: {c.area:.2f}")
print(f"Circumference: {c.circumference:.2f}")

# Computed properties update automatically
c.radius = 10
print(f"\nNew radius: {c.radius}")
print(f"New area: {c.area:.2f}")

In [ ]:
# Property with caching (lazy evaluation)
class ExpensiveComputation:
    def __init__(self, data):
        self.data = data
        self._cached_result = None
    
    @property
    def result(self):
        if self._cached_result is None:
            print("Computing expensive result...")
            # Simulate expensive computation
            self._cached_result = sum(x ** 2 for x in self.data)
        return self._cached_result
    
    def invalidate_cache(self):
        """Call when data changes."""
        self._cached_result = None

comp = ExpensiveComputation([1, 2, 3, 4, 5])

print(f"First access: {comp.result}")   # Computes
print(f"Second access: {comp.result}")  # Cached
print(f"Third access: {comp.result}")   # Cached

comp.invalidate_cache()
print(f"After invalidate: {comp.result}")  # Recomputes

In [ ]:
# Property for data transformation
class Temperature:
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = (value - 32) * 5/9
    
    @property
    def kelvin(self):
        return self._celsius + 273.15
    
    @kelvin.setter
    def kelvin(self, value):
        self.celsius = value - 273.15

t = Temperature(25)
print(f"25C = {t.fahrenheit}F = {t.kelvin}K")

t.fahrenheit = 98.6  # Set via Fahrenheit
print(f"{t.fahrenheit}F = {t.celsius:.1f}C")

t.kelvin = 300  # Set via Kelvin
print(f"{t.kelvin}K = {t.celsius:.1f}C")

---

## 6. Class Decorators

Decorators can also be applied to entire classes.

In [ ]:
# Class decorator to add functionality
def add_repr(cls):
    """Add automatic __repr__ to class."""
    def __repr__(self):
        attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{cls.__name__}({attrs})"
    cls.__repr__ = __repr__
    return cls

@add_repr
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

@add_repr
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

print(Point(3, 4))
print(Person("Alice", 30))

In [ ]:
# Singleton class decorator
def singleton(cls):
    """Make class a singleton."""
    instances = {}
    
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance

@singleton
class Database:
    def __init__(self, connection_string):
        print(f"Connecting to {connection_string}")
        self.connection_string = connection_string

db1 = Database("localhost:5432")
db2 = Database("different:5432")  # Won't create new instance

print(f"Same instance: {db1 is db2}")
print(f"Connection: {db2.connection_string}")

In [ ]:
# Class decorator with arguments
def dataclass_like(frozen=False):
    """Simple dataclass-like decorator."""
    def decorator(cls):
        # Add __repr__
        def __repr__(self):
            attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
            return f"{cls.__name__}({attrs})"
        cls.__repr__ = __repr__
        
        # Add __eq__
        def __eq__(self, other):
            if not isinstance(other, cls):
                return NotImplemented
            return self.__dict__ == other.__dict__
        cls.__eq__ = __eq__
        
        # If frozen, prevent attribute modification
        if frozen:
            original_setattr = cls.__setattr__ if hasattr(cls, '__setattr__') else object.__setattr__
            cls._initialized = False
            
            def __setattr__(self, name, value):
                if hasattr(self, '_frozen') and self._frozen:
                    raise AttributeError(f"Cannot modify frozen instance")
                original_setattr(self, name, value)
            
            original_init = cls.__init__
            def new_init(self, *args, **kwargs):
                original_init(self, *args, **kwargs)
                object.__setattr__(self, '_frozen', True)
            
            cls.__setattr__ = __setattr__
            cls.__init__ = new_init
        
        return cls
    return decorator

@dataclass_like(frozen=True)
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p1 = Point(3, 4)
p2 = Point(3, 4)

print(f"p1: {p1}")
print(f"p1 == p2: {p1 == p2}")

try:
    p1.x = 10  # Frozen!
except AttributeError as e:
    print(f"Error: {e}")

In [ ]:
# Using dataclasses (built-in since Python 3.7)
from dataclasses import dataclass, field

@dataclass
class Product:
    name: str
    price: float
    quantity: int = 0
    
    @property
    def total_value(self):
        return self.price * self.quantity

@dataclass(frozen=True)  # Immutable
class Point:
    x: int
    y: int

# Auto-generated __init__, __repr__, __eq__
p = Product("Widget", 9.99, 100)
print(f"Product: {p}")
print(f"Total value: ${p.total_value:.2f}")

pt = Point(3, 4)
print(f"Point: {pt}")

---

## 7. Decorating Methods with Arguments

In [ ]:
# Decorator with arguments for methods
from functools import wraps

def repeat(times):
    """Decorator that repeats method execution."""
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            results = []
            for _ in range(times):
                results.append(func(self, *args, **kwargs))
            return results
        return wrapper
    return decorator

class Greeter:
    @repeat(times=3)
    def greet(self, name):
        return f"Hello, {name}!"

g = Greeter()
print(g.greet("Alice"))

In [ ]:
# Retry decorator with parameters
import random
from functools import wraps

def retry(max_attempts=3, exceptions=(Exception,)):
    """Retry method on failure."""
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            last_exception = None
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(self, *args, **kwargs)
                except exceptions as e:
                    last_exception = e
                    print(f"Attempt {attempt} failed: {e}")
            raise last_exception
        return wrapper
    return decorator

class UnreliableAPI:
    def __init__(self):
        self.call_count = 0
    
    @retry(max_attempts=5, exceptions=(ConnectionError,))
    def fetch_data(self):
        self.call_count += 1
        # Simulate unreliable connection
        if random.random() < 0.7:  # 70% failure rate
            raise ConnectionError("Connection failed")
        return "Data fetched successfully!"

api = UnreliableAPI()
try:
    result = api.fetch_data()
    print(f"Result: {result}")
except ConnectionError:
    print("All attempts failed")
print(f"Total attempts: {api.call_count}")

---

## 8. Method Decorator Use Cases

In [ ]:
# Authorization decorator
from functools import wraps

def require_permission(permission):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            if not hasattr(self, 'permissions'):
                raise PermissionError("No permissions defined")
            if permission not in self.permissions:
                raise PermissionError(f"Permission '{permission}' required")
            return func(self, *args, **kwargs)
        return wrapper
    return decorator

class SecureResource:
    def __init__(self, permissions):
        self.permissions = permissions
    
    @require_permission('read')
    def read_data(self):
        return "Sensitive data"
    
    @require_permission('write')
    def write_data(self, data):
        return f"Written: {data}"
    
    @require_permission('admin')
    def delete_all(self):
        return "Everything deleted"

# User with limited permissions
limited = SecureResource(['read'])
print(limited.read_data())

try:
    limited.write_data("test")
except PermissionError as e:
    print(f"Error: {e}")

# Admin user
admin = SecureResource(['read', 'write', 'admin'])
print(admin.write_data("admin data"))
print(admin.delete_all())

In [ ]:
# Validation decorator
from functools import wraps

def validate_args(**validators):
    """Validate method arguments."""
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            # Get argument names
            import inspect
            sig = inspect.signature(func)
            params = list(sig.parameters.keys())[1:]  # Skip 'self'
            
            # Combine args and kwargs
            bound_args = dict(zip(params, args))
            bound_args.update(kwargs)
            
            # Validate
            for param, validator in validators.items():
                if param in bound_args:
                    value = bound_args[param]
                    if not validator(value):
                        raise ValueError(f"Invalid value for '{param}': {value}")
            
            return func(self, *args, **kwargs)
        return wrapper
    return decorator

class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
    
    @validate_args(amount=lambda x: x > 0)
    def deposit(self, amount):
        self.balance += amount
        return self.balance
    
    @validate_args(amount=lambda x: x > 0)
    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        return self.balance

account = BankAccount(100)
print(f"Deposit $50: Balance = ${account.deposit(50)}")
print(f"Withdraw $30: Balance = ${account.withdraw(30)}")

try:
    account.deposit(-50)  # Invalid
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
# Deprecation decorator
import warnings
from functools import wraps

def deprecated(message="This method is deprecated"):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            warnings.warn(
                f"{func.__name__}: {message}",
                DeprecationWarning,
                stacklevel=2
            )
            return func(self, *args, **kwargs)
        return wrapper
    return decorator

class Calculator:
    def add(self, a, b):
        return a + b
    
    @deprecated("Use 'add' instead")
    def plus(self, a, b):
        return self.add(a, b)

calc = Calculator()
print(calc.add(3, 5))

# This will show a deprecation warning
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    result = calc.plus(3, 5)
    print(f"Result: {result}")
    if w:
        print(f"Warning: {w[0].message}")

---

## 9. Combining Multiple Decorators

In [ ]:
# Multiple decorators - execution order
from functools import wraps

def decorator_a(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("A: Before")
        result = func(*args, **kwargs)
        print("A: After")
        return result
    return wrapper

def decorator_b(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("B: Before")
        result = func(*args, **kwargs)
        print("B: After")
        return result
    return wrapper

class Example:
    @decorator_a  # Applied second (outer)
    @decorator_b  # Applied first (inner)
    def method(self):
        print("Method executing")

# Equivalent to: method = decorator_a(decorator_b(method))
e = Example()
e.method()

In [ ]:
# Practical example: combining logging, timing, and authorization
from functools import wraps
import time

def log_call(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        print(f"[LOG] Calling {func.__name__}")
        return func(self, *args, **kwargs)
    return wrapper

def time_call(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        start = time.time()
        result = func(self, *args, **kwargs)
        print(f"[TIME] {func.__name__}: {time.time() - start:.4f}s")
        return result
    return wrapper

def require_auth(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        if not getattr(self, 'authenticated', False):
            raise PermissionError("Authentication required")
        return func(self, *args, **kwargs)
    return wrapper

class SecureService:
    def __init__(self, authenticated=False):
        self.authenticated = authenticated
    
    @log_call
    @time_call
    @require_auth
    def sensitive_operation(self):
        time.sleep(0.1)  # Simulate work
        return "Operation completed"

# Authenticated user
service = SecureService(authenticated=True)
print(service.sensitive_operation())

print()

# Unauthenticated user
service2 = SecureService(authenticated=False)
try:
    service2.sensitive_operation()
except PermissionError as e:
    print(f"Error: {e}")

---

## 10. Practical Examples

In [ ]:
# Complete example: API client with decorators
from functools import wraps
import time

def rate_limit(calls_per_second):
    """Limit method calls per second."""
    min_interval = 1.0 / calls_per_second
    last_called = [0.0]
    
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            elapsed = time.time() - last_called[0]
            wait_time = min_interval - elapsed
            if wait_time > 0:
                print(f"Rate limiting: waiting {wait_time:.2f}s")
                time.sleep(wait_time)
            last_called[0] = time.time()
            return func(self, *args, **kwargs)
        return wrapper
    return decorator

def cache_response(ttl_seconds):
    """Cache method results for specified time."""
    cache = {}
    
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            key = (args, tuple(sorted(kwargs.items())))
            if key in cache:
                result, timestamp = cache[key]
                if time.time() - timestamp < ttl_seconds:
                    print(f"Cache hit for {args}")
                    return result
            print(f"Cache miss for {args}")
            result = func(self, *args, **kwargs)
            cache[key] = (result, time.time())
            return result
        return wrapper
    return decorator

class APIClient:
    def __init__(self, base_url):
        self.base_url = base_url
    
    @rate_limit(calls_per_second=2)
    @cache_response(ttl_seconds=5)
    def get_user(self, user_id):
        """Simulated API call."""
        return {"id": user_id, "name": f"User{user_id}"}

client = APIClient("https://api.example.com")

# First call - cache miss
print(f"Result: {client.get_user(1)}")
print()

# Second call - cache hit
print(f"Result: {client.get_user(1)}")
print()

# Different user - cache miss, may rate limit
print(f"Result: {client.get_user(2)}")

In [ ]:
# Complete example: Model class with validations
from functools import wraps

class Field:
    """Descriptor for validated fields."""
    def __init__(self, validator=None):
        self.validator = validator
        self.name = None
    
    def __set_name__(self, owner, name):
        self.name = name
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)
    
    def __set__(self, obj, value):
        if self.validator and not self.validator(value):
            raise ValueError(f"Invalid value for {self.name}: {value}")
        obj.__dict__[self.name] = value

def positive(value):
    return isinstance(value, (int, float)) and value > 0

def non_empty_string(value):
    return isinstance(value, str) and len(value.strip()) > 0

def valid_email(value):
    import re
    return bool(re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', value))

class User:
    name = Field(validator=non_empty_string)
    email = Field(validator=valid_email)
    age = Field(validator=positive)
    
    def __init__(self, name, email, age):
        self.name = name
        self.email = email
        self.age = age
    
    def __repr__(self):
        return f"User('{self.name}', '{self.email}', {self.age})"

# Valid user
user = User("Alice", "alice@example.com", 25)
print(f"Valid user: {user}")

# Invalid cases
for args, desc in [
    (("", "test@test.com", 25), "empty name"),
    (("Bob", "invalid-email", 30), "invalid email"),
    (("Charlie", "c@c.com", -5), "negative age")
]:
    try:
        User(*args)
    except ValueError as e:
        print(f"Error ({desc}): {e}")

---

## 11. Key Points

**Built-in Method Decorators:**
- `@staticmethod`: No implicit first argument, utility functions
- `@classmethod`: Receives `cls`, factory methods, class state
- `@property`: Managed attributes with getters/setters

**Custom Method Decorators:**
- Include `self` as first parameter in wrapper
- Use `@functools.wraps` to preserve metadata
- Can accept arguments using nested functions

**Class Decorators:**
- Receive and return the class
- Can add methods, modify behavior
- Used for singleton, dataclass patterns

**Decorator Order:**
- Applied bottom to top
- Executed top to bottom (outer first)

**Common Use Cases:**
- Logging, timing, caching
- Authorization, validation
- Retry logic, rate limiting
- Deprecation warnings

---

## 12. Practice Exercises

In [ ]:
# Exercise 1: Create a class with multiple constructors
# - Rectangle class with width and height
# - @classmethod from_square(cls, side) - creates square
# - @classmethod from_string(cls, dims_string) - parses "WxH"
# - @property area and perimeter
# - @staticmethod is_valid_dimensions(w, h)

class Rectangle:
    pass

# Test:
# r1 = Rectangle(3, 4)
# r2 = Rectangle.from_square(5)
# r3 = Rectangle.from_string("6x8")
# print(r1.area, r2.perimeter)

In [ ]:
# Exercise 2: Create a type_check decorator
# - Takes type hints as arguments
# - Validates method arguments at runtime
# - Raises TypeError if type mismatch

def type_check(**types):
    pass

# Test:
# class Calculator:
#     @type_check(a=int, b=int)
#     def add(self, a, b):
#         return a + b
# calc = Calculator()
# calc.add(1, 2)  # OK
# calc.add("1", 2)  # TypeError

In [ ]:
# Exercise 3: Create a count_calls class decorator
# - Adds call counting to all public methods
# - Adds get_call_counts() method to return dict of counts

def count_calls(cls):
    pass

# Test:
# @count_calls
# class Service:
#     def method1(self): pass
#     def method2(self): pass
# s = Service()
# s.method1(); s.method1(); s.method2()
# print(s.get_call_counts())  # {'method1': 2, 'method2': 1}

In [ ]:
# Exercise 4: Create a lazy_property decorator
# - Computes value on first access
# - Caches result as instance attribute
# - Subsequent access returns cached value

class lazy_property:
    pass

# Test:
# class Data:
#     @lazy_property
#     def computed(self):
#         print("Computing...")
#         return sum(range(1000000))
# d = Data()
# print(d.computed)  # Computing... <result>
# print(d.computed)  # <result> (no computing)

In [ ]:
# Exercise 5: Create a debug_methods class decorator
# - Wraps all methods to print entry/exit and args/return
# - Skip private methods (starting with _)
# - Include timing information

def debug_methods(cls):
    pass

# Test:
# @debug_methods
# class Calculator:
#     def add(self, a, b): return a + b
#     def _internal(self): pass  # Skipped
# calc = Calculator()
# calc.add(3, 5)

---

## Solutions

In [ ]:
# Solution 1:
class Rectangle:
    def __init__(self, width, height):
        if not Rectangle.is_valid_dimensions(width, height):
            raise ValueError("Dimensions must be positive")
        self._width = width
        self._height = height
    
    def __repr__(self):
        return f"Rectangle({self._width}, {self._height})"
    
    @classmethod
    def from_square(cls, side):
        return cls(side, side)
    
    @classmethod
    def from_string(cls, dims_string):
        width, height = map(float, dims_string.lower().split('x'))
        return cls(width, height)
    
    @property
    def area(self):
        return self._width * self._height
    
    @property
    def perimeter(self):
        return 2 * (self._width + self._height)
    
    @staticmethod
    def is_valid_dimensions(w, h):
        return w > 0 and h > 0

r1 = Rectangle(3, 4)
r2 = Rectangle.from_square(5)
r3 = Rectangle.from_string("6x8")

print(f"r1: {r1}, area={r1.area}, perimeter={r1.perimeter}")
print(f"r2 (square): {r2}, area={r2.area}")
print(f"r3 (from string): {r3}")

In [ ]:
# Solution 2:
from functools import wraps
import inspect

def type_check(**types):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            # Get parameter names
            sig = inspect.signature(func)
            params = list(sig.parameters.keys())[1:]  # Skip self
            
            # Bind arguments
            bound = dict(zip(params, args))
            bound.update(kwargs)
            
            # Check types
            for param, expected_type in types.items():
                if param in bound:
                    value = bound[param]
                    if not isinstance(value, expected_type):
                        raise TypeError(
                            f"Argument '{param}' must be {expected_type.__name__}, "
                            f"got {type(value).__name__}"
                        )
            return func(self, *args, **kwargs)
        return wrapper
    return decorator

class Calculator:
    @type_check(a=int, b=int)
    def add(self, a, b):
        return a + b
    
    @type_check(value=(int, float))
    def square(self, value):
        return value ** 2

calc = Calculator()
print(f"add(1, 2) = {calc.add(1, 2)}")
print(f"square(3.5) = {calc.square(3.5)}")

try:
    calc.add("1", 2)
except TypeError as e:
    print(f"Error: {e}")

In [ ]:
# Solution 3:
from functools import wraps

def count_calls(cls):
    cls._call_counts = {}
    
    def get_call_counts(self):
        return dict(self._call_counts)
    
    cls.get_call_counts = get_call_counts
    
    for name, method in list(cls.__dict__.items()):
        if callable(method) and not name.startswith('_'):
            cls._call_counts[name] = 0
            
            def make_wrapper(method_name, original):
                @wraps(original)
                def wrapper(self, *args, **kwargs):
                    self._call_counts[method_name] += 1
                    return original(self, *args, **kwargs)
                return wrapper
            
            setattr(cls, name, make_wrapper(name, method))
    
    return cls

@count_calls
class Service:
    def method1(self):
        return "method1 called"
    
    def method2(self):
        return "method2 called"
    
    def _private(self):
        return "private"

s = Service()
s.method1()
s.method1()
s.method2()
s._private()  # Not counted

print(f"Call counts: {s.get_call_counts()}")

In [ ]:
# Solution 4:
class lazy_property:
    def __init__(self, func):
        self.func = func
        self.attr_name = None
    
    def __set_name__(self, owner, name):
        self.attr_name = name
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.attr_name not in obj.__dict__:
            obj.__dict__[self.attr_name] = self.func(obj)
        return obj.__dict__[self.attr_name]

class Data:
    def __init__(self, n):
        self.n = n
    
    @lazy_property
    def computed(self):
        print("Computing expensive operation...")
        return sum(range(self.n))
    
    @lazy_property
    def squared(self):
        print("Computing squares...")
        return [x**2 for x in range(self.n)]

d = Data(1000000)
print(f"First access: {d.computed}")
print(f"Second access: {d.computed}")
print(f"Third access: {d.computed}")

In [ ]:
# Solution 5:
from functools import wraps
import time

def debug_methods(cls):
    for name, method in list(cls.__dict__.items()):
        if callable(method) and not name.startswith('_'):
            def make_wrapper(method_name, original):
                @wraps(original)
                def wrapper(self, *args, **kwargs):
                    args_str = ', '.join(
                        [repr(a) for a in args] + 
                        [f"{k}={v!r}" for k, v in kwargs.items()]
                    )
                    print(f"[ENTER] {method_name}({args_str})")
                    start = time.time()
                    try:
                        result = original(self, *args, **kwargs)
                        elapsed = time.time() - start
                        print(f"[EXIT] {method_name} -> {result!r} ({elapsed:.4f}s)")
                        return result
                    except Exception as e:
                        elapsed = time.time() - start
                        print(f"[ERROR] {method_name} raised {type(e).__name__}: {e} ({elapsed:.4f}s)")
                        raise
                return wrapper
            setattr(cls, name, make_wrapper(name, method))
    return cls

@debug_methods
class Calculator:
    def add(self, a, b):
        return a + b
    
    def divide(self, a, b):
        return a / b
    
    def _internal(self):
        return "internal"

calc = Calculator()
calc.add(3, 5)
print()
calc.divide(10, 2)
print()

try:
    calc.divide(1, 0)
except ZeroDivisionError:
    pass